In [143]:
import requests
import json
import re
from bs4 import BeautifulSoup
import wikitextparser as wtp

In [144]:
# Fandom API 網址
API_URL = "https://wutheringwaves.fandom.com/api.php"

In [145]:
# 取得所有可操作角色 (Playable Resonators)
def GetCharacterList():
    params = {
        "action": "query",
        "list": "categorymembers",
        "cmtitle": "Category:Echo_Stats",
        "cmlimit": "100",  # 限制最多 100 個角色
        "format": "json"
    }
    response = requests.get(API_URL, params = params)
    data = response.json()
    
    # 解析角色名稱列表
    characters = [char["title"] for char in data["query"]["categorymembers"]]
    return characters

In [146]:
# 取得單個角色的詳細 Wiki 內容
def GetCharacterData(characterName):
    params = {
        "action": "query",
        "titles": characterName,
        "prop": "revisions",
        "rvprop": "content",
        "format": "json"
    }
    response = requests.get(API_URL, params = params)
    data = response.json()
    
    # 取得 wikitext 內容
    pages = data["query"]["pages"]
    pageContent = next(iter(pages.values()))["revisions"][0]["*"]
    
    return pageContent

In [147]:
def CharacterMain():
    characters = GetCharacterList()
    #print(f"找到 {len(characters)} 位角色：", characters)

    allCharacterData = []
    
    for character in characters:
        #print(f"正在處理角色：{character} ...")
        
        # 獲取角色詳細資訊
        wikiText = GetCharacterData(character)
        
        # 解析成 JSON
        characterInfo = DecodingWikiText(wikiText)
        allCharacterData.append(characterInfo)
    
    #print("\n角色資訊整理完成！")
    print(allCharacterData)

In [148]:
def GetEchoStats():
    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "titles": "Echoes/Stats",
        "rvprop": "content"
    }
    response = requests.get(API_URL, params=params)
    data = response.json()

    pages = data["query"]["pages"]
    pageContent = next(iter(pages.values()))["revisions"][0]["*"]
    
    return pageContent

In [149]:
def EchoMain():
    echoStats = GetEchoStats()
    #print(f"找到 {len(characters)} 位角色：", characters)

    echoData = wtp.parse(echoStats)
    
    #echoDataJson = json.loads(echoData)

    #echoData = ParseWikiText(echoStats["Detailed substat values distribution"])
    
    print(json.dumps(echoData.tables[3].data(), indent=4, ensure_ascii=False))


In [150]:
if __name__ == "__main__":
    EchoMain()

[
    [
        "'''Substat'''",
        "'''Value1'''",
        "'''Value2'''",
        "'''Value3'''",
        "'''Value4'''"
    ],
    [
        "[[ATK]]",
        "30",
        "40",
        "50",
        "60"
    ],
    [
        "[[DEF]]",
        "40",
        "50",
        "60",
        "70"
    ],
    [
        "'''Chance to roll'''",
        "12.43%",
        "46.21%",
        "38.57%",
        "2.79%"
    ]
]
